In [3]:
# Find RL_Note path and append sys path
import os, sys
cwd = os.getcwd()
dir_name = 'RL_note'
pos = cwd.find(dir_name)
root_path = cwd[0:pos] + dir_name
sys.path.append(root_path)
print(root_path)
workspace_path = root_path + "\\pys"

c:\Users\User\RL_note


In [4]:
import gym
import random
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, Conv2DTranspose, Flatten, Dropout, BatchNormalization, Reshape, LeakyReLU, ReLU
from tensorflow.keras.losses import MeanSquaredError, BinaryCrossentropy
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint

from pys.utils.er import ReplayMemory
from pys.utils.per import ProportionalPrioritizedMemory
from pys.utils.her import HindsightMemory
from pys.model.q_network import QNetwork

In [ ]:
def get_encoder(input_shape, compressed_shape):
    X_input = Input(shape=input_shape, name='Input')
    X = X_input
    X = Conv2D(filters=32, kernel_size=8, strides=4, padding='same', \
                data_format="channels_last", name='Encoder1')(X)
    X = BatchNormalization(name="BN1", axis=1)(X)
    X = LeakyReLU(name='LReLU1')(X)

    X = Conv2D(filters=64, kernel_size=8, strides=4, padding='same', \
                data_format="channels_last", name='Encoder2')(X)
    X = BatchNormalization(name="BN2", axis=1)(X)
    X = LeakyReLU(name='LReLU2')(X)

    X = Conv2D(filters=64, kernel_size=3, strides=2, padding='same', \
                data_format="channels_last", name='Encoder3')(X)
    X = BatchNormalization(name="BN3", axis=1)(X)
    X = LeakyReLU(name='LReLU3')(X)

    X = Conv2D(filters=64, kernel_size=3, strides=1, padding='same', \
                data_format="channels_last", name='Encoder4')(X)
    X = BatchNormalization(name="BN4", axis=1)(X)
    X = LeakyReLU(name='LReLU4')(X)
    # X = MaxPool2D(filters-

    X = Flatten(name='Flattening')(X)

    encoder_output = Dense(compressed_shape[0], activation='linear', \
                name='EncoderOut')(X)
    encoder_model = Model(inputs=X_input, outputs=encoder_output, name='Encoder')

    return encoder_model

def get_decoder(input_shape, compressed_shape):
    X_input = Input(shape=compressed_shape, name='Input')
    X = X_input

    X = Dense(units=7*7*64, activation='linear', \
                name='DeFlattening')(X)
    X = Reshape((7,7,64), name='Reshape')(X)

    X = Conv2DTranspose(filters=64, kernel_size=3, strides=1, padding='same', \
                data_format="channels_last", name='Decoder1')(X)
    X = BatchNormalization(name="BN1", axis=1)(X)
    X = LeakyReLU(name='LReLU1')(X)

    X = Conv2DTranspose(filters=64, kernel_size=3, strides=2, padding='same', \
                data_format="channels_last", name='Decoder2')(X)
    X = BatchNormalization(name="BN2", axis=1)(X)
    X = LeakyReLU(name='LReLU2')(X)

    X = Conv2DTranspose(filters=64, kernel_size=3, strides=2, padding='same', \
                data_format="channels_last", name='Decoder3')(X)
    X = BatchNormalization(name="BN3", axis=1)(X)
    X = LeakyReLU(name='LReLU3')(X)

    X = Conv2DTranspose(filters=32, kernel_size=3, strides=1, padding='same', \
                data_format="channels_last", name='Decoder4')(X)
    X = BatchNormalization(name="BN4", axis=1)(X)
    X = LeakyReLU(name='LReLU4')(X)

    decoder_output = Conv2DTranspose(filters=1, kernel_size=3, strides=1, padding='same', \
                data_format="channels_last", name='Decoder_out', activation='tanh')(X)
    decoder_model = Model(inputs=X_input, outputs=decoder_output, name='Decoder')
    
    return decoder_model

def AutoEncoder(input_shape, compressed_shape):
        input_shape = input_shape
        compressed_shape = compressed_shape

        # Hyper-parameter
        learning_rate = 0.001
        batch_size = 32

        # Define auto-encoder
        encoder = get_encoder(input_shape=input_shape, compressed_shape=compressed_shape)
        decoder = get_decoder(input_shape=input_shape, compressed_shape=compressed_shape)
        # encoder.compile(optimizer=Adam(learning_rate=learning_rate), loss=MeanSquaredError())
        # decoder.compile(optimizer=Adam(learning_rate=learning_rate), loss=MeanSquaredError())
        # encoder.summary()
        # decoder.summary()

        # Connect encoder with decoder
        encoder_in = Input(shape=input_shape)
        decoder_out= decoder(encoder(encoder_in))
        
        auto_encoder = Model(inputs=encoder_in, outputs=decoder_out)
        auto_encoder.compile(optimizer=Adam(learning_rate=learning_rate), loss=MeanSquaredError())
        # auto_encoder.compile(optimizer=Adam(learning_rate=learning_rate), loss=BinaryCrossentropy())
        auto_encoder.summary()
        return auto_encoder, encoder, decoder

In [ ]:
class DQNAgent:
    def __init__(self, env:object, cfg:dict):
        self.state_size = (160,240)
        self.action_size= env.action_space.n
        self.env_name   = cfg["ENV"]
        self.rl_type    = "DQN"
        self.er_type    = cfg["ER"]["ALGORITHM"].upper()
        self.filename   = cfg["ENV"] + '_' + cfg["RL"]["ALGORITHM"] + '_' + cfg["ER"]["ALGORITHM"]

        # Experience Replay
        self.batch_size = cfg["BATCH_SIZE"]
        self.train_start = cfg["TRAIN_START"]
        self.buffer_size = cfg["MEMORY_SIZE"]
        if self.er_type == "ER":
            self.memory = ReplayMemory(capacity=self.buffer_size)
        elif self.er_type == "PER":
            self.memory = ProportionalPrioritizedMemory(capacity=self.buffer_size)
        elif self.er_type == "HER":
            self.memory = HindsightMemory(\
                capacity            = self.buffer_size,\
                replay_n            = cfg["ER"]["REPLAY_N"],\
                replay_strategy     = cfg["ER"]["STRATEGY"],\
                reward_func         = cfg["ER"]["REWARD_FUNC"],\
                done_func           = cfg["ER"]["DONE_FUNC"])
            self.filename = cfg["ENV"] + '_' + cfg["RL"]["ALGORITHM"] + '_' + cfg["ER"]["ALGORITHM"] + '_' + cfg["ER"]["STRATEGY"]

        # Hyper-parameters for learning
        self.discount_factor = 0.99
        self.learning_rate = 0.001
        self.epsilon = 1.0
        self.epsilon_decay = 0.999
        self.epsilon_min = 0.01
        self.tau = 0.005
        
        # Neural Network Architecture
        self.compressed_shape = (4,)
        self.auto_encoder, self.encoder, self.decoder = AutoEncoder(self.state_size, self.compressed_shape)
        self.model        = QNetwork(self.state_size, self.action_size, cfg["RL"]["NETWORK"])
        self.target_model = QNetwork(self.state_size, self.action_size, cfg["RL"]["NETWORK"])
        self.optimizer = tf.keras.optimizers.Adam(lr=self.learning_rate)
        self.hard_update_target_model()
        
        # Miscellaneous
        self.show_media_info = False
        self.steps = 0
        self.update_period = 100
        # self.interaction_period = 1

        print(self.filename)
        print('States {0}, Actions {1}'.format(self.state_size, self.action_size))
        
    def remember(self, state, action, reward, next_state, done, goal=None):
        state       = np.array(state,       dtype=np.float32)
        action      = np.array([action])
        reward      = np.array([reward],    dtype=np.float32)
        done        = np.array([done],      dtype=np.float32)
        next_state  = np.array(next_state,  dtype=np.float32)
        if self.er_type == "HER":
            goal        = np.array(goal,        dtype=np.float32)
            transition  = (state, action, reward, next_state, done, goal)
        else:
            transition  = (state, action, reward, next_state, done)
        self.memory.append(transition)
        return
        
    def hard_update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    def soft_update_target_model(self):
        tau = self.tau
        for (net, target_net) in zip(   self.model.trainable_variables,
                                        self.target_model.trainable_variables):
            target_net.assign(tau * net + (1.0 - tau) * target_net)

    def get_action(self,state):
        self.steps += 1
        # Exploration and Exploitation
        if (np.random.rand() <= self.epsilon):
            return random.randrange(self.action_size)
        else:
            state = tf.convert_to_tensor([state], dtype=tf.float32)
            return np.argmax(self.model(state))
        
    def train_model(self):
        # Train from Experience Replay
        # Training Condition - Memory Size
        if len(self.memory) < self.train_start:
            return 0.0
        # Decaying Exploration Ratio
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
        # Sampling from the memory
        if self.er_type == "ER" or self.er_type == "HER":
            mini_batch = self.memory.sample(self.batch_size)
        elif self.er_type == "PER":
            mini_batch, idxs, is_weights = self.memory.sample(self.batch_size)

        states      = tf.convert_to_tensor(np.array([sample[0] for sample in mini_batch]))
        actions     = tf.convert_to_tensor(np.array([sample[1][0] for sample in mini_batch]))
        rewards     = tf.convert_to_tensor(np.array([sample[2] for sample in mini_batch]))
        next_states = tf.convert_to_tensor(np.array([sample[3] for sample in mini_batch]))
        dones       = tf.convert_to_tensor(np.array([sample[4] for sample in mini_batch]))
        
        if self.show_media_info == False:
            self.show_media_info = True
            print('Start to train, check batch shapes')
            print('**** shape of mini_batch', np.shape(mini_batch),type(mini_batch))
            print('**** shape of states', np.shape(states),type(states))
            print('**** shape of actions', np.shape(actions),type(actions))
            print('**** shape of rewards', np.shape(rewards),type(rewards))
            print('**** shape of next_states', np.shape(next_states),type(next_states))
            print('**** shape of dones', np.shape(dones),type(dones))
            if self.er_type == "HER":
                goals = tf.convert_to_tensor(np.array([sample[5] for sample in mini_batch]))
                print('**** shape of goals', np.shape(goals),type(goals))

        model_params = self.model.trainable_variables
        with tf.GradientTape() as tape:
            # get q value
            q = self.model(states)
            one_hot_action = tf.one_hot(actions, self.action_size)
            q = tf.reduce_sum(one_hot_action * q, axis=1)
            q = tf.expand_dims(q,axis=1)
            # Target q and maximum target q
            target_q = tf.stop_gradient(self.target_model(next_states))
            max_q = tf.reduce_max(target_q,axis=1)
            max_q = tf.expand_dims(max_q,axis=1)
            
            targets = rewards + (1 - dones) * self.discount_factor * max_q
            td_error = targets - q
            loss = tf.reduce_mean(tf.square(targets - q))
            
        grads = tape.gradient(loss, model_params)
        self.optimizer.apply_gradients(zip(grads, model_params))

        if self.er_type == "PER":
            sample_importance = td_error.numpy()
            for i in range(self.batch_size):
                self.memory.update(idxs[i], sample_importance[i])

        return loss

    def step_update(self):
        if self.steps % self.update_period != 0:
            self.soft_update_target_model()
        return

    def episode_update(self):
        checkpoint_path = '01_basic_auto_encoder_' + data + '.ckpt'
        if (training==True):
            # Train
            checkpoint = ModelCheckpoint(checkpoint_path, 
                                        save_best_only=True, 
                                        save_weights_only=True, 
                                        monitor='loss', 
                                        verbose=1)
            auto_encoder.fit(x_train, x_train, 
                            batch_size=batch_size, 
                            epochs=100, 
                            callbacks=[checkpoint], 
                            )
            auto_encoder.save_weights(checkpoint_path)
            print('Save model weights')
        else:
            auto_encoder.load_weights(checkpoint_path)
            print('Load model weights')
        pass

    def load_model(self,at:str):
        self.model.load_weights( at + self.filename + "_TF")
        self.target_model.load_weights(at + self.filename + "_TF")
        return

    def save_model(self,at:str):
        self.model.save_weights( at + self.filename + "_TF", save_format="tf")
        self.target_model.save_weights(at + self.filename + "_TF", save_format="tf")
        return

In [ ]:
if __name__ == "__main__":
    cfg = {\
            # "ENV":"Pong-v0",\
            "ENV":"CartPole-v1",\
            "RL":{
                "ALGORITHM":"DQN",\
                "NETWORK":{
                    "LAYER":[255,255],\

                }
            },\
            "ER":
                {
                    "ALGORITHM":"ER",\
                    "REPLAY_N":8,\
                    "STRATEGY":"EPISODE",\
                    # "REWARD_FUNC":reward_function,\
                    # "DONE_FUNC":done_function,\
                },\
            "BATCH_SIZE":8,\
            "TRAIN_START":1000,\
            "MEMORY_SIZE":20000,\
            }
    env_config = env_configs[cfg["ENV"]]
    if cfg["ER"]["ALGORITHM"] == "HER":
        FILENAME = cfg["ENV"] + '_' + cfg["RL"]["ALGORITHM"] + '_' + cfg["ER"]["ALGORITHM"] + '_' + cfg["HER"]["STRATEGY"]
    else:
        FILENAME = cfg["ENV"] + '_' + cfg["RL"]["ALGORITHM"] + '_' + cfg["ER"]["ALGORITHM"]
    EPISODES = env_config["EPISODES"]
    END_SCORE = env_config["END_SCORE"]

    env = gym.make(cfg["ENV"])
    agent = DQNAgent(env, cfg)

    plt.clf()
    figure = plt.gcf()
    figure.set_size_inches(8,6)

    scores_avg, scores_raw, episodes, losses = [], [], [], []
    epsilons = []
    score_avg = 0
    end = False
    show_media_info = True
    goal = np.array([1.0,0.0,0.0])
    
    for e in range(EPISODES):
        # Episode initialization
        done = False
        score = 0
        loss_list = []
        state = env.reset()
        while not done:
            # env.render()
            # Interact with env.
            action = agent.get_action(state)
            next_state, reward, done, info = env.step(action)
            agent.remember(state, action, reward, next_state, done, goal)
            loss = agent.train_model()
            agent.step_update()
            state = next_state
            # 
            score += reward
            loss_list.append(loss)
            if show_media_info:
                print("-------------- Variable shapes --------------")
                print("State Shape : ", np.shape(state))
                print("Action Shape : ", np.shape(action))
                print("Reward Shape : ", np.shape(reward))
                print("done Shape : ", np.shape(done))
                print("---------------------------------------------")
                show_media_info = False
            if done:
                agent.episode_update()
                score_avg = 0.9 * score_avg + 0.1 * score if score_avg != 0 else score
                print("episode: {0:3d} | score avg: {1:3.2f} | mem size {2:6d} |"
                    .format(e, score_avg, len(agent.memory)))

                episodes.append(e)
                scores_avg.append(score_avg)
                scores_raw.append(score)
                losses.append(np.mean(loss_list))
                epsilons.append(agent.epsilon)
                # View data
                plt.clf()
                plt.subplot(311)
                plt.plot(episodes, scores_avg, 'b')
                plt.plot(episodes, scores_raw, 'b', alpha=0.8, linewidth=0.5)
                plt.xlabel('episode'); plt.ylabel('average score'); plt.grid()
                plt.title(FILENAME)
                plt.subplot(312)
                plt.plot(episodes, epsilons, 'b')
                plt.xlabel('episode'); plt.ylabel('epsilon'); plt.grid()
                plt.subplot(313)
                plt.plot(episodes, losses, 'b')
                plt.xlabel('episode'); plt.ylabel('losses') ;plt.grid()
                # plt.savefig(workspace_path + "\\result\\img\\" + FILENAME + "_TF.jpg", dpi=100)

                # 이동 평균이 0 이상일 때 종료
                if score_avg > END_SCORE:
                    # agent.save_model(workspace_path + "\\result\\save_model\\")
                    end = True
                    break
        if end == True:
            env.close()
            # np.save(workspace_path + "\\result\\data\\" + FILENAME + "_TF_epi",  episodes)
            # np.save(workspace_path + "\\result\\data\\" + FILENAME + "_TF_scores_avg",scores_avg)
            # np.save(workspace_path + "\\result\\data\\" + FILENAME + "_TF_scores_raw",scores_raw)
            # np.save(workspace_path + "\\result\\data\\" + FILENAME + "_TF_losses",losses)
            print("End")
            break